# 1. 오픈데이터 설명

### 1. 서울시 경찰서별 5대 범죄 발생 및 검거 현황 데이터
* 파일명: ["경찰청 서울특별시경찰청_경찰서별 5대 범죄 발생 검거 현황_20221231.csv"](https://www.data.go.kr/data/15054738/fileData.do)
* 내용: 2022년 서울시 각 경찰서별로 5대 범죄(강간, 강도, 살인, 절도, 폭력)의 발생 건수와 검거 건수
#### 주요 항목
* 경찰서별: 경찰서 명칭 (예: 중부, 종로, 남대문 등)
* 죄종: 범죄 유형 (예: 강간, 강도, 살인, 절도, 폭력)
* 발생 검거: '발생' 또는 '검거' 여부
* 건수: 해당 죄종의 발생 또는 검거 건수

### 2. 서울시 주민등록 인구 데이터
* 파일명: ["202212_202212_주민등록인구 및 세대 현황_연간.csv"](https://jumin.mois.go.kr/)
* 내용: 2022년 서울시 각 행정구역별 주민등록 인구수
#### 주요 항목:
* 행정구역: 행정구역 명칭 (예: 서울특별시 종로구 등)
* 2022년_총인구수: 해당 구역의 남녀 합산 인구수
* 2022년_세대수: 해당 구역의 세대수
* 2022년_세대당 인구: 총인구수 / 세대수
* 2022년_남자 인구수: 남자 인구수
* 2022년_여자 인구수: 여자 인구수
* 2022년_남녀 비율: 남자 인구수 / 여자 인구수

### 3. 분석 사유
[“가해자 처벌 강화해라”…강남 한복판 60대男 칼부림으로 모녀 사망](https://www.topstarnews.net/news/articleView.html?idxno=15501728)\
위의 기사는 최근에 일어난 강남 살인사건에 대한 기사입니다. 우리나라는 세계적으로도 치안이 우수한 나라로 평가받고 있음에도 불구하고, 이러한 비극적인 사건이 발생한다는 것은 많은 이들에게 충격과 함께 큰 우려를 불러일으키는 일입니다. 사회의 안정성과 개인의 안전을 위협하는 이러한 문제들은 우리 주변에서 일어날 수 있기 때문입니다.

서울은 대한민국의 수도로, 다양한 문화, 경제 활동이 집중되어 있는 곳이지만, 그만큼 다양한 사회적 문제들도 포함하고 있습니다. 이 기사를 읽은 저는 한 가지 의문이 생겼습니다. "우리 사회의 치안은 정말 우리가 생각하는 만큼 안전한가?" 서울 내 각 구별 인구당 5대 범죄율과 구별 5대 범죄 검거율에 대한 분석을 통해 이 의문에 답변이 될 수 있을 것 같아 데이터 분석을 진행하게 되었습니다.

### 4. 분석 목표
#### "서울시 2022년 경찰서별 5대 범죄 발생 및 검거 현황과 주민등록 인구 데이터를 분석해 서울 내 각 구별 인구당 5대 범죄율과 구별 5대 범죄 검거율을 파악"

# 2. 데이터 보기

In [1]:
## 판다스 라이브러리 불러오기
import pandas as pd

### 1. 범죄 현황 데이터

In [2]:
## 서울시 5대 범죄 데이터 불러오기 @ data.go.kr
## read csv() 함수로 데이터 프레임 변환
crime = pd.read_csv('경찰청 서울특별시경찰청_경찰서별 5대 범죄 발생 검거 현황_20221231.csv', encoding='cp949')

## 데이터 백업
crime.to_csv('crime_backup.csv')

## 상위 5개 데이터 출력
crime.head()

,구분,죄종,발생검거,건수
0,중부,살인,발생,1
1,중부,살인,검거,2
2,중부,강도,발생,3
3,중부,강도,검거,2
4,중부,"강간,추행",발생,137


#### 범죄 현황 데이터를 깔끔한 데이터로 만들기 위해 데이터 전처리

In [3]:
## 새로운 형식의 데이터 프레임을 위한 딕셔너리 초기화
result = {
    '경찰서': [],
    '발생 소계': [],
    '검거 소계': [],
    '살인 발생': [],
    '살인 검거': [],
    '강도 발생': [],
    '강도 검거': [],
    '강간 발생': [],
    '강간 검거': [],
    '절도 발생': [],
    '절도 검거': [],
    '폭력 발생': [],
    '폭력 검거': []
}

## 경찰서명 추출
police_stations = crime['구분'].unique()

## 경찰서별로 데이터 정리
for station in police_stations:
    result['경찰서'].append(station)
    
    ## 발생, 검거 소계 초기화
    total_incidents = crime[(crime['구분'] == station) & (crime['발생검거'] == '발생')]['건수'].sum()
    total_arrests = crime[(crime['구분'] == station) & (crime['발생검거'] == '검거')]['건수'].sum()
    
    result['발생 소계'].append(total_incidents)
    result['검거 소계'].append(total_arrests)
    
    ## 각 죄종별 발생, 검거 건수 초기화
    for crime_type in ['강간', '강도', '살인', '절도', '폭력']:
        incidents = crime[(crime['구분'] == station) & (crime['죄종'].str.contains(crime_type)) & (crime['발생검거'] == '발생')]['건수'].sum()
        arrests = crime[(crime['구분'] == station) & (crime['죄종'].str.contains(crime_type)) & (crime['발생검거'] == '검거')]['건수'].sum()
        
        ## 결과 딕셔너리에 추가
        result[f'{crime_type} 발생'].append(incidents)
        result[f'{crime_type} 검거'].append(arrests)

## 결과 데이터프레임 생성
crime = pd.DataFrame(result)
crime.head()

,경찰서,발생 소계,검거 소계,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부,2077,1389,1,2,3,2,137,87,910,459,1026,839
1,종로,1719,1178,2,-1,5,4,142,120,723,333,847,722
2,남대문,993,729,-1,-1,2,2,57,36,455,265,480,427
3,서대문,2374,1605,5,4,6,6,157,144,1114,565,1092,886
4,혜화,1418,943,-1,-1,3,3,86,60,590,283,740,598


In [4]:
# 범죄 데이터의 기본 정보 확인
crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   경찰서     31 non-null     object
 1   발생 소계   31 non-null     int64 
 2   검거 소계   31 non-null     int64 
 3   살인 발생   31 non-null     int64 
 4   살인 검거   31 non-null     int64 
 5   강도 발생   31 non-null     int64 
 6   강도 검거   31 non-null     int64 
 7   강간 발생   31 non-null     int64 
 8   강간 검거   31 non-null     int64 
 9   절도 발생   31 non-null     int64 
 10  절도 검거   31 non-null     int64 
 11  폭력 발생   31 non-null     int64 
 12  폭력 검거   31 non-null     int64 
dtypes: int64(12), object(1)
memory usage: 3.3+ KB


In [5]:
## 범죄 데이터의 기술 통계 출력
## 범죄 발생 및 검거 현황인데 -1이 나온 건 이상 데이터이므로 데이터 클렌징 과정에서 0으로 변경할 필요가 있음
crime.describe()

,발생 소계,검거 소계,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,2915.903226,2062.225806,3.290323,2.967742,3.645161,3.451613,187.612903,146.322581,1212.225806,641.580645,1509.129032,1267.903226
std,1236.935740,846.012636,2.673245,2.549299,2.994978,2.987069,109.847069,88.766881,512.339452,251.154637,658.905139,546.639330
min,757.000000,589.000000,-1.000000,-1.000000,-1.000000,-1.000000,55.000000,36.000000,305.000000,188.000000,391.000000,355.000000
25%,1922.000000,1370.000000,2.000000,2.000000,2.000000,2.000000,108.500000,84.500000,821.500000,473.000000,967.000000,809.000000
50%,2832.000000,2093.000000,2.000000,2.000000,3.000000,3.000000,157.000000,121.000000,1118.000000,589.000000,1566.000000,1252.000000
75%,3784.000000,2657.500000,5.000000,4.000000,5.000000,6.000000,245.500000,185.000000,1565.000000,833.500000,1996.500000,1695.000000
max,5167.000000,3591.000000,11.000000,11.000000,10.000000,9.000000,458.000000,368.000000,2201.000000,1185.000000,2669.000000,2201.000000


### 2. 인구수 데이터

In [6]:
## 서울시 주민등록인구 데이터 불러오기 @ jumin.mois.go.kr
## read csv() 함수로 데이터 프레임 변환
population = pd.read_csv('202212_202212_주민등록인구 및 세대 현황_연간.csv', encoding='cp949')

## 데이터 백업
population.to_csv('population_backup.csv')

population

,행정구역,2022년_총인구수,2022년_세대수,2022년_세대당 인구,2022년_남자 인구수,2022년_여자 인구수,2022년_남여 비율
0,서울특별시 (1100000000),"9,428,372","4,446,296",2.12,"4,570,048","4,858,324",0.94
1,서울특별시 서울특별시 종로구 (1111000000),"141,379","72,524",1.95,"68,395","72,984",0.94
2,서울특별시 서울특별시 중구 (1114000000),"120,437","63,139",1.91,"58,563","61,874",0.95
3,서울특별시 서울특별시 용산구 (1117000000),"218,650","109,805",1.99,"105,087","113,563",0.93
4,서울특별시 서울특별시 성동구 (1120000000),"281,000","133,305",2.11,"136,633","144,367",0.95
5,서울특별시 서울특별시 광진구 (1121500000),"337,416","169,291",1.99,"162,541","174,875",0.93
6,서울특별시 서울특별시 동대문구 (1123000000),"336,644","169,873",1.98,"165,755","170,889",0.97
7,서울특별시 서울특별시 중랑구 (1126000000),"385,318","187,413",2.06,"189,537","195,781",0.97
8,서울특별시 서울특별시 성북구 (1129000000),"430,397","197,082",2.18,"206,961","223,436",0.93
9,서울특별시 서울특별시 강북구 (1130500000),"293,660","144,313",2.03,"142,567","151,093",0.94


#### 데이터 분석에 필요한 자료(구이름, 전체 인구 수)만 추출

In [7]:
## 행정구역에서 구 이름 추출
population['구별'] = population['행정구역'].apply(lambda x: x.split()[-2])

## 필요한 열만 선택
population = population[['구별', '2022년_총인구수']]

## 열 이름 변경
population.columns = ['구별', '인구수']

## 첫 번째 행(서울특별시 전체 인구 데이터) 삭제
population = population.drop(0)

## 구별 데이터로 변경
population.set_index('구별', inplace=True)

## 인구수 데이터 타입을 int로 변경 -> 범죄 데이터 타입과 통일
population['인구수'] = population['인구수'].str.replace(',', '').astype(int)

population

,인구수
구별,
종로구,141379
중구,120437
용산구,218650
성동구,281000
광진구,337416
동대문구,336644
중랑구,385318
성북구,430397
강북구,293660


In [8]:
## 인구 데이터 정보 확인
population.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 종로구 to 강동구
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   인구수     25 non-null     int64
dtypes: int64(1)
memory usage: 400.0+ bytes


In [9]:
## 인구 데이터의 기술 통계 출력
population.describe()

,인구수
count,25.000000
mean,377134.880000
std,127071.592274
min,120437.000000
25%,306337.000000
50%,380596.000000
75%,460067.000000
max,658801.000000


# 3. 데이터 클린징

In [10]:
## 범죄 데이터의 중 0 미만의 값(이상 데이터) 0으로 변경
crime = crime.applymap(lambda x: 0 if isinstance(x, int) and x < 0 else x)
crime

/var/folders/6_/81pvr66d2l779qybvtywvshm0000gn/T/ipykernel_36032/1435546663.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  crime = crime.applymap(lambda x: 0 if isinstance(x, int) and x < 0 else x)


,경찰서,발생 소계,검거 소계,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부,2077,1389,1,2,3,2,137,87,910,459,1026,839
1,종로,1719,1178,2,0,5,4,142,120,723,333,847,722
2,남대문,993,729,0,0,2,2,57,36,455,265,480,427
3,서대문,2374,1605,5,4,6,6,157,144,1114,565,1092,886
4,혜화,1418,943,0,0,3,3,86,60,590,283,740,598
5,용산,2967,2093,5,3,7,7,280,238,978,508,1697,1337
6,성북,1375,1066,1,1,0,0,103,78,540,335,732,653
7,동대문,3253,2231,4,4,2,2,125,106,1556,823,1566,1296
8,마포,4096,2814,2,2,2,1,436,368,1451,653,2205,1790
9,영등포,4819,3186,5,6,10,9,327,237,2062,953,2415,1981


# 4. 데이터 탐색

### 1. 경찰서별 데이터 구별로 그룹화

In [11]:
# 경찰서 위치 정보 데이터 불러오기
police_to_gu = {
    '서대문': '서대문구', '수서': '강남구', '강서': '강서구', '서초': '서초구',
    '서부': '은평구', '중부': '중구', '종로': '종로구', '남대문': '중구',
    '혜화': '종로구', '용산': '용산구', '성북': '성북구', '동대문': '동대문구',
    '마포': '마포구', '영등포': '영등포구', '성동': '성동구', '동작': '동작구',
    '광진': '광진구', '강북': '강북구', '금천': '금천구', '중랑': '중랑구',
    '강남': '강남구', '관악': '관악구', '강동': '강동구', '종암': '성북구',
    '구로': '구로구', '양천': '양천구', '송파': '송파구', '노원': '노원구',
    '방배': '서초구', '은평': '은평구', '도봉': '도봉구'
}

# 경찰서명을 구별로 변경하여 새로운 열 '구별' 생성
crime['구별'] = crime['경찰서'].apply(lambda x: police_to_gu.get(x, '구 없음'))

crime.head()

,경찰서,발생 소계,검거 소계,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부,2077,1389,1,2,3,2,137,87,910,459,1026,839,중구
1,종로,1719,1178,2,0,5,4,142,120,723,333,847,722,종로구
2,남대문,993,729,0,0,2,2,57,36,455,265,480,427,중구
3,서대문,2374,1605,5,4,6,6,157,144,1114,565,1092,886,서대문구
4,혜화,1418,943,0,0,3,3,86,60,590,283,740,598,종로구


In [12]:
## 구별로 그루핑하여 범죄 발생, 검거 건수 합계 계산
crime = crime.groupby('구별').sum()

## 사용하지 않는 경찰서 열 삭제
crime.drop('경찰서', axis=1, inplace=True)
crime

,발생 소계,검거 소계,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
구별,,,,,,,,,,,,
강남구,6947,5020,12,10,11,10,667,534,2495,1207,3762,3259
강동구,3711,2644,5,6,5,5,157,134,1672,900,1872,1599
강북구,2832,2343,5,4,5,6,177,147,872,589,1773,1597
강서구,4663,3591,3,3,7,7,261,195,1991,1185,2401,2201
관악구,4879,3411,10,8,9,8,327,279,1966,1050,2567,2066
광진구,3618,2497,6,5,0,0,230,174,1670,831,1713,1488
구로구,3857,2732,11,11,8,9,217,145,1565,841,2056,1726
금천구,2577,1887,2,1,3,3,127,87,1118,662,1327,1134
노원구,3896,2641,3,3,3,1,180,151,1504,729,2206,1757


### 2. 검거율 합성 변수 만들고, 불필요한 데이터 삭제하기

In [13]:
## 발생건수 대비 검거건수 -> 검거율 데이터를 범죄별로 생성
crime['강간검거율'] = crime['강간 검거']/crime['강간 발생']*100
crime['강도검거율'] = crime['강도 검거']/crime['강도 발생']*100
crime['살인검거율'] = crime['살인 검거']/crime['살인 발생']*100
crime['절도검거율'] = crime['절도 검거']/crime['절도 발생']*100
crime['폭력검거율'] = crime['폭력 검거']/crime['폭력 발생']*100
crime['검거율'] = crime['검거 소계']/crime['발생 소계']*100
crime

,발생 소계,검거 소계,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율,검거율
구별,,,,,,,,,,,,,,,,,,
강남구,6947,5020,12,10,11,10,667,534,2495,1207,3762,3259,80.059970,90.909091,83.333333,48.376754,86.629452,72.261408
강동구,3711,2644,5,6,5,5,157,134,1672,900,1872,1599,85.350318,100.000000,120.000000,53.827751,85.416667,71.247642
강북구,2832,2343,5,4,5,6,177,147,872,589,1773,1597,83.050847,120.000000,80.000000,67.545872,90.073322,82.733051
강서구,4663,3591,3,3,7,7,261,195,1991,1185,2401,2201,74.712644,100.000000,100.000000,59.517830,91.670137,77.010508
관악구,4879,3411,10,8,9,8,327,279,1966,1050,2567,2066,85.321101,88.888889,80.000000,53.407935,80.483054,69.911867
광진구,3618,2497,6,5,0,0,230,174,1670,831,1713,1488,75.652174,NaN,83.333333,49.760479,86.865149,69.016031
구로구,3857,2732,11,11,8,9,217,145,1565,841,2056,1726,66.820276,112.500000,100.000000,53.738019,83.949416,70.832253
금천구,2577,1887,2,1,3,3,127,87,1118,662,1327,1134,68.503937,100.000000,50.000000,59.212880,85.455916,73.224680
노원구,3896,2641,3,3,3,1,180,151,1504,729,2206,1757,83.888889,33.333333,100.000000,48.470745,79.646419,67.787474


In [14]:
## 미 사용 열 지우기 (범죄별 발생 건수와 검거율만 남김)
crime = crime.drop(['발생 소계', '검거 소계', '강간 검거', '강도 검거', '살인 검거', '절도 검거', '폭력 검거'], axis=1)
crime

,살인 발생,강도 발생,강간 발생,절도 발생,폭력 발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율,검거율
구별,,,,,,,,,,,
강남구,12,11,667,2495,3762,80.059970,90.909091,83.333333,48.376754,86.629452,72.261408
강동구,5,5,157,1672,1872,85.350318,100.000000,120.000000,53.827751,85.416667,71.247642
강북구,5,5,177,872,1773,83.050847,120.000000,80.000000,67.545872,90.073322,82.733051
강서구,3,7,261,1991,2401,74.712644,100.000000,100.000000,59.517830,91.670137,77.010508
관악구,10,9,327,1966,2567,85.321101,88.888889,80.000000,53.407935,80.483054,69.911867
광진구,6,0,230,1670,1713,75.652174,NaN,83.333333,49.760479,86.865149,69.016031
구로구,11,8,217,1565,2056,66.820276,112.500000,100.000000,53.738019,83.949416,70.832253
금천구,2,3,127,1118,1327,68.503937,100.000000,50.000000,59.212880,85.455916,73.224680
노원구,3,3,180,1504,2206,83.888889,33.333333,100.000000,48.470745,79.646419,67.787474


In [15]:
## 열 이름을 새롭게 정의
crime.rename(columns = {'강간 발생':'강간',
                     '강도 발생':'강도',
                     '살인 발생':'살인',
                     '절도 발생':'절도',
                     '폭력 발생':'폭력'}, inplace=True) # inplace 옵션 == 덮어쓰기 여부

crime

,살인,강도,강간,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율,검거율
구별,,,,,,,,,,,
강남구,12,11,667,2495,3762,80.059970,90.909091,83.333333,48.376754,86.629452,72.261408
강동구,5,5,157,1672,1872,85.350318,100.000000,120.000000,53.827751,85.416667,71.247642
강북구,5,5,177,872,1773,83.050847,120.000000,80.000000,67.545872,90.073322,82.733051
강서구,3,7,261,1991,2401,74.712644,100.000000,100.000000,59.517830,91.670137,77.010508
관악구,10,9,327,1966,2567,85.321101,88.888889,80.000000,53.407935,80.483054,69.911867
광진구,6,0,230,1670,1713,75.652174,NaN,83.333333,49.760479,86.865149,69.016031
구로구,11,8,217,1565,2056,66.820276,112.500000,100.000000,53.738019,83.949416,70.832253
금천구,2,3,127,1118,1327,68.503937,100.000000,50.000000,59.212880,85.455916,73.224680
노원구,3,3,180,1504,2206,83.888889,33.333333,100.000000,48.470745,79.646419,67.787474


### 3. 합성 변수(검거율) 데이터 클린징

In [16]:
## 검거율이 100이 넘는 경우 100으로 맞춤
crime[crime[['강간검거율', '강도검거율', '살인검거율', '절도검거율', '폭력검거율']] > 100 ] = 100
crime

,살인,강도,강간,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율,검거율
구별,,,,,,,,,,,
강남구,12,11,667,2495,3762,80.059970,90.909091,83.333333,48.376754,86.629452,72.261408
강동구,5,5,157,1672,1872,85.350318,100.000000,100.000000,53.827751,85.416667,71.247642
강북구,5,5,177,872,1773,83.050847,100.000000,80.000000,67.545872,90.073322,82.733051
강서구,3,7,261,1991,2401,74.712644,100.000000,100.000000,59.517830,91.670137,77.010508
관악구,10,9,327,1966,2567,85.321101,88.888889,80.000000,53.407935,80.483054,69.911867
광진구,6,0,230,1670,1713,75.652174,NaN,83.333333,49.760479,86.865149,69.016031
구로구,11,8,217,1565,2056,66.820276,100.000000,100.000000,53.738019,83.949416,70.832253
금천구,2,3,127,1118,1327,68.503937,100.000000,50.000000,59.212880,85.455916,73.224680
노원구,3,3,180,1504,2206,83.888889,33.333333,100.000000,48.470745,79.646419,67.787474


In [17]:
## 검거율에서 NaN이 발생한 이유는 범죄의 발생이 0이기 때문
## 검거가 1이상 이라면 0으로 채우면 데이터의 정확한 분석이 불가능하나 검거도 0인걸 확인했기 때문에 이번에 한해서만 NaN 값을 0으로 채움
crime.fillna(0, inplace=True)
crime

,살인,강도,강간,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율,검거율
구별,,,,,,,,,,,
강남구,12,11,667,2495,3762,80.059970,90.909091,83.333333,48.376754,86.629452,72.261408
강동구,5,5,157,1672,1872,85.350318,100.000000,100.000000,53.827751,85.416667,71.247642
강북구,5,5,177,872,1773,83.050847,100.000000,80.000000,67.545872,90.073322,82.733051
강서구,3,7,261,1991,2401,74.712644,100.000000,100.000000,59.517830,91.670137,77.010508
관악구,10,9,327,1966,2567,85.321101,88.888889,80.000000,53.407935,80.483054,69.911867
광진구,6,0,230,1670,1713,75.652174,0.000000,83.333333,49.760479,86.865149,69.016031
구로구,11,8,217,1565,2056,66.820276,100.000000,100.000000,53.738019,83.949416,70.832253
금천구,2,3,127,1118,1327,68.503937,100.000000,50.000000,59.212880,85.455916,73.224680
노원구,3,3,180,1504,2206,83.888889,33.333333,100.000000,48.470745,79.646419,67.787474


### 4. 범죄 현황 데이터, 인구수 데이터 병합

In [18]:
## 범죄 데이터와 인구 데이터를 합침
df = pd.merge(crime, population, how='outer', on='구별')
df

,살인,강도,강간,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율,검거율,인구수
구별,,,,,,,,,,,,
강남구,12,11,667,2495,3762,80.059970,90.909091,83.333333,48.376754,86.629452,72.261408,529102
강동구,5,5,157,1672,1872,85.350318,100.000000,100.000000,53.827751,85.416667,71.247642,460067
강북구,5,5,177,872,1773,83.050847,100.000000,80.000000,67.545872,90.073322,82.733051,293660
강서구,3,7,261,1991,2401,74.712644,100.000000,100.000000,59.517830,91.670137,77.010508,569166
관악구,10,9,327,1966,2567,85.321101,88.888889,80.000000,53.407935,80.483054,69.911867,486752
광진구,6,0,230,1670,1713,75.652174,0.000000,83.333333,49.760479,86.865149,69.016031,337416
구로구,11,8,217,1565,2056,66.820276,100.000000,100.000000,53.738019,83.949416,70.832253,395315
금천구,2,3,127,1118,1327,68.503937,100.000000,50.000000,59.212880,85.455916,73.224680,229642
노원구,3,3,180,1504,2206,83.888889,33.333333,100.000000,48.470745,79.646419,67.787474,503734


In [19]:
## 검거율 기준으로 오름차순 정렬하기
df.sort_values(by='검거율', ascending=False, inplace=True)
df

,살인,강도,강간,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율,검거율,인구수
구별,,,,,,,,,,,,
강북구,5,5,177,872,1773,83.050847,100.000000,80.000000,67.545872,90.073322,82.733051,293660
성북구,3,1,158,1143,1444,81.645570,100.000000,100.000000,62.554681,88.434903,77.292576,430397
강서구,3,7,261,1991,2401,74.712644,100.000000,100.000000,59.517830,91.670137,77.010508,569166
은평구,4,5,176,1497,1805,82.954545,80.000000,100.000000,63.193053,86.315789,76.225982,466746
중랑구,2,2,150,1508,1937,80.666667,100.000000,100.000000,58.488064,85.906040,74.215060,385318
도봉구,3,0,70,969,1099,77.142857,0.000000,100.000000,56.346749,87.898089,73.271028,311694
금천구,2,3,127,1118,1327,68.503937,100.000000,50.000000,59.212880,85.455916,73.224680,229642
성동구,2,2,125,979,1086,77.600000,100.000000,100.000000,54.954035,88.213628,72.789426,281000
강남구,12,11,667,2495,3762,80.059970,90.909091,83.333333,48.376754,86.629452,72.261408,529102


# 5. 데이터 분석

### 각 구별 인구당 5대 범죄율을 계산

In [20]:
## 5대 범죄별 최대건수
weight_col = df[['강간', '강도', '살인', '절도', '폭력']].max()
weight_col

강간     667
강도      11
살인      12
절도    2495
폭력    3762
dtype: int64

In [21]:
## 정규화된 범죄 발생 건수 데이터프레임 생성
crime_count_norm = df[['강간', '강도', '살인', '절도', '폭력']] / weight_col
crime_count_norm

,강간,강도,살인,절도,폭력
구별,,,,,
강북구,0.265367,0.454545,0.416667,0.349499,0.471292
성북구,0.236882,0.090909,0.250000,0.458116,0.383838
강서구,0.391304,0.636364,0.250000,0.797996,0.638224
은평구,0.263868,0.454545,0.333333,0.600000,0.479798
중랑구,0.224888,0.181818,0.166667,0.604409,0.514886
도봉구,0.104948,0.000000,0.250000,0.388377,0.292132
금천구,0.190405,0.272727,0.166667,0.448096,0.352738
성동구,0.187406,0.181818,0.166667,0.392385,0.288676
강남구,1.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
## 행(구)별로 구별 범죄 수 (max 대비 비율값) / 구별 인구 수 * 100000 * 100
## 인구 수 단위인 10만을 곱해줌 (강서구 강간 = 9.795665e-07 -> 0.x 까지 끌어올리기)
## 단위를 %로 변환하기 위해 100을 추가로 곱해줌

crime_ratio = crime_count_norm.div(df['인구수'], axis=0) * 100000 * 100
crime_ratio

,강간,강도,살인,절도,폭력
구별,,,,,
강북구,9.036550,15.478630,14.188744,11.901485,16.048896
성북구,5.503792,2.112215,5.808591,10.644039,8.918240
강서구,6.875048,11.180633,4.392392,14.020444,11.213325
은평구,5.653355,9.738604,7.141643,12.854958,10.279638
중랑구,5.836414,4.718653,4.325432,15.685974,13.362617
도봉구,3.367005,0.000000,8.020687,12.460193,9.372392
금천구,8.291375,11.876193,7.257674,19.512815,15.360339
성동구,6.669263,6.470398,5.931198,13.963871,10.273176
강남구,18.899947,18.899947,18.899947,18.899947,18.899947


In [23]:
## 구별 인구 대비 전체 범죄 발생 비율
crime_ratio['전체발생비율'] = crime_ratio.mean(axis=1) ## 정확한 비율은 아니지만 상대적인 심각도를 판단하는데는 큰 차이가 없을것으로 판단함
crime_ratio

,강간,강도,살인,절도,폭력,전체발생비율
구별,,,,,,
강북구,9.036550,15.478630,14.188744,11.901485,16.048896,13.330861
성북구,5.503792,2.112215,5.808591,10.644039,8.918240,6.597375
강서구,6.875048,11.180633,4.392392,14.020444,11.213325,9.536368
은평구,5.653355,9.738604,7.141643,12.854958,10.279638,9.133639
중랑구,5.836414,4.718653,4.325432,15.685974,13.362617,8.785818
도봉구,3.367005,0.000000,8.020687,12.460193,9.372392,6.644056
금천구,8.291375,11.876193,7.257674,19.512815,15.360339,12.459679
성동구,6.669263,6.470398,5.931198,13.963871,10.273176,8.661581
강남구,18.899947,18.899947,18.899947,18.899947,18.899947,18.899947


In [24]:
## 결과 저장
crime_ratio.to_csv('result.csv', sep=',', encoding='utf-8')

# 6. 데이터 시각화

### folium 라이브러리 사용법
* [참고자료1](https://velog.io/@eodud0582/Folium)
* [참고자료2](https://velog.io/@zer0/EDA-Folium-%EC%A7%80%EB%8F%84-%EC%8B%9C%EA%B0%81%ED%99%94)

In [25]:
import json ## json 라이브러리 불러오기
import folium ## 지도 시각화 라이브러리 불러오기
from shapely.geometry import shape ## 지도 데이터를 다루기 위한 라이브러리 불러오기

In [26]:
geo_path = 'skorea_municipalities_geo_simple.json' ## 서울시의 행정구역 경계 정보가 담겨 있는 GeoJSON 파일
geo_str = json.load(open(geo_path, encoding='utf-8')) ## GeoJSON 파일 불러오기 

### 1. 각 구별 인구당 5대 범죄율 시각화

In [27]:
# 지도 타입과 초기 위치 설정 -> location: 서울시청의 위도, 경도
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner', attr='Attribution Text')

## 전체 발생 비율 시각화
folium.Choropleth(
    geo_data = geo_str,  ## 서울시 행정구역별 폴리곤 드로잉
    data = crime_ratio['전체발생비율'],  ## 시각화 대상 데이터
    columns = [crime_ratio.index, crime_ratio['전체발생비율']],  ## 데이터 프레임의 인덱스 및 '전체발생비율' 열 사용
    fill_color = 'PuRd',  ## 색상 설정 (예: PuRd, YlGnBu 등 color brewer 사용)
    key_on = 'feature.id',  ## GeoJSON 파일의 'feature.id'에 데이터 바인딩
    legend_name = '10만 명당 5대 범죄율 (%)' ## 범례 이름
).add_to(map) ## map에 추가

## GeoJSON 파일의 각 행정구역 데이터를 기반으로 중심 좌표를 계산해 구 이름을 지도에 표시
for feature in geo_str['features']:
    polygon = shape(feature['geometry'])  ## 'geometry' 정보를 사용하여 구 도형을 생성
    centroid = polygon.centroid  ## 생성된 구 도형 중심점을 계산
    text = f"{feature['id']}"  # 구 이름 문자열 형태로 추출

    
    folium.map.Marker(
        location = [centroid.y, centroid.x], ## 문자를 적을 좌표 -> 구의 중심 좌표
        icon=folium.DivIcon(
            html=f"""
                <div style="
                    font-size: 12px;
                    color: black;
                    white-space: nowrap;
                ">
                    {text}
                </div>
            """ ## html 문법 사용 가능
        )
    ).add_to(map)

map

### 2. 각 구별 5대 범죄 검거율 시각화

In [28]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner', attr='Attribution Text')

# 검거율 시각화
folium.Choropleth(
    geo_data= geo_str,
    data = crime['검거율'],
    columns = [crime.index, crime['검거율']],
    fill_color = 'YlGnBu',
    key_on='feature.id',
    legend_name= '5대 범죄 검거율 (%)'
).add_to(map)

## GeoJSON 파일의 각 행정구역 데이터를 기반으로 중심 좌표를 계산해 구 이름을 지도에 표시
for feature in geo_str['features']:
    polygon = shape(feature['geometry'])
    centroid = polygon.centroid
    text = f"{feature['id']}"

    folium.map.Marker(
        location = [centroid.y, centroid.x],
        icon=folium.DivIcon(
            html=f"""
                <div style="
                    font-size: 12px;
                    color: black;
                    white-space: nowrap;
                    transform: translate(-50%, -50%);
                ">
                    {text}
                </div>
            """
        )
    ).add_to(map)

map

### 3. 시각화 결과 분석

### 인구당 범죄율 대비 검거율이 낮은 2개 구
종로구, 중구
### 인구당 범죄율 대비 검거율이 높은 2개 구
강서구, 강북구
### 결론
종로구, 중구 등 범죄율 대비 검거율이 낮은 지역들은 검거 인력을 보충해 검거율을 높이고 범죄율 하락을 위한 순찰 강하를 시행해야 합니다. 인력이 부족하면, 강서구, 강북구 등 범죄율 대비 검거율이 높은 지역들의 인적 지원을 받는 것도 방법이지만, 과도한 업무로 인해 기존의 검거율을 낮추는 결과는 가져오지 않도록 해야 합니다.
기대한 우리나라의 검거율을 80% 이상이었으나 그러지 못하는 결과를 보고 놀랐습니다. 다른 나라는 어떤지 잘 모르지만, 국민들의 안전을 위해 검거율을 높일 수 있도록 방안을 구상할 필요는 있다고 생각합니다.

# 7. 데이터 또는 분석 과정 차별화 포인트 

1. 더러운 범죄자 데이터를 정제하여 깔끔한 데이터로 변환
2. 경찰서와 구를 매핑하는 테이블을 만들어 구별로 데이터를 그룹화
3. 검거율 합성 변수를 생성
4. 범죄 발생 건수를 정규화하고 folium 라이브러리를 사용하여 데이터를 시각화

# 8. 데이터분석 수업을 마감하며 

한 학기 동안 데이터 분석 수업을 들으면서, 저는 파이썬의 다양한 라이브러리를 활용하여 데이터를 분석하는 방법을 배웠습니다. 처음에는 코드가 낯설고 어려웠지만, 꾸준히 공부하면서 이해도를 높일 수 있었고, 이제는 간단한 데이터 분석을 스스로 할 수 있을 정도로 성장했습니다. 이는 제게 큰 원동력이 되었습니다.

또한, 이영호 교수님께서 현업에 대한 이야기를 많이 해주셔서 매우 유익한 인사이트를 얻을 수 있었습니다. 교수님의 경험을 바탕으로 제공해주신 현실적인 조언과 사례들은 이론을 실제 사례와 연결하는 데 큰 도움이 되었습니다.

특히, 현업자분들과의 인터뷰 과제를 통해 실제 현업에서 활동하시는 분들과 소통하는 경험을 처음 하게 되었는데, 이를 통해 현업과의 소통이 얼마나 중요한지 깊이 깨달았습니다. 현업분들의 이야기를 들으면서 많은 것을 배울 수 있었고, 이는 제 학습에 큰 자극이 되었습니다.

이번 인터뷰를 계기로 연을 맺은 분들과 앞으로도 지속적으로 소통하면서, 저의 인사이트를 더 넓혀가고 싶습니다. 이 과정을 통해 현업에서의 경험을 더 많이 배우고, 실제로 적용할 수 있는 능력을 더욱 키우고자 합니다. 이러한 지속적인 교류는 제 개인적인 성장뿐만 아니라, 저의 실질적인 역량 강화에도 큰 도움이 될 것이라고 믿습니다.

앞으로도 지금 느낀 것들을 바탕으로 더욱 성장하는 컴퓨터 공학도가 되겠습니다.